In [1]:
import json, random, shutil, os
from pathlib import Path

# --- Paths (edit these two to match your structure) ---
JSON_PATH   = Path("data/grounded_reports_20240819 (1).json")
IMAGES_DIR  = Path("data/PadChest_GR")  # or "data/PadChest_GR_resized_512_512"
OUT_ROOT    = Path("datasets/padchest_yolo")  # output dataset root

# Split ratios
SPLITS = {"train": 0.8, "val": 0.1, "test": 0.1}
random.seed(42)

# --- Load JSON ---
with open(JSON_PATH, "r", encoding="utf-8") as f:
    items = json.load(f)

# --- Collect labels (classes) from findings ---
label_set = set()
for it in items:
    for fnd in it.get("findings", []):
        for lab in fnd.get("labels", []):
            label_set.add(lab.strip())

# Make stable, deterministic ordering
names = sorted(label_set)
name_to_id = {n: i for i, n in enumerate(names)}

# --- Prepare output dirs ---
for split in ["train","val","test"]:
    (OUT_ROOT / "images" / split).mkdir(parents=True, exist_ok=True)
    (OUT_ROOT / "labels" / split).mkdir(parents=True, exist_ok=True)

# --- Helper: convert x1y1x2y2 (normalized) -> xc,yc,w,h (normalized) ---
def to_yolo(x1, y1, x2, y2):
    xc = (x1 + x2) / 2.0
    yc = (y1 + y2) / 2.0
    w  = (x2 - x1)
    h  = (y2 - y1)
    return xc, yc, w, h

# --- Build list of available images from JSON that actually exist ---
records = []
for it in items:
    img_name = it.get("ImageID")
    if not img_name:
        continue
    src_img = IMAGES_DIR / img_name
    if not src_img.is_file():
        # silently skip images that are not in the chosen images folder
        continue

    # Build all YOLO bboxes for this image
    yolo_lines = []
    for fnd in it.get("findings", []):
        boxes = fnd.get("boxes", []) or []
        labels = fnd.get("labels", []) or []

        # Decide the class label to use:
        # - If multiple labels are present, use the FIRST (customize if needed)
        # - If no labels, skip the finding
        cls_name = labels[0].strip() if labels else None
        if cls_name is None or cls_name not in name_to_id:
            continue
        cls_id = name_to_id[cls_name]

        for b in boxes:
            if len(b) != 4:
                continue
            x1, y1, x2, y2 = map(float, b)
            # Skip degenerate boxes
            if x2 <= x1 or y2 <= y1:
                continue
            xc, yc, w, h = to_yolo(x1, y1, x2, y2)
            # keep only boxes that are inside [0,1]
            if not (0 <= xc <= 1 and 0 <= yc <= 1 and 0 < w <= 1 and 0 < h <= 1):
                continue
            yolo_lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

    # keep images that have at least one box
    if yolo_lines:
        records.append((src_img, yolo_lines))

# --- Train/Val/Test split ---
random.shuffle(records)
n = len(records)
n_train = int(n * SPLITS["train"])
n_val   = int(n * SPLITS["val"])
splits = {
    "train": records[:n_train],
    "val":   records[n_train:n_train+n_val],
    "test":  records[n_train+n_val:],
}

# --- Write files ---
for split, recs in splits.items():
    for src_img, yolo_lines in recs:
        dst_img = OUT_ROOT / "images" / split / src_img.name
        dst_lab = OUT_ROOT / "labels" / split / (src_img.stem + ".txt")

        # copy image
        shutil.copy2(src_img, dst_img)
        # write label txt
        with open(dst_lab, "w", encoding="utf-8") as f:
            f.write("\n".join(yolo_lines))

# --- Create data.yaml ---
yaml_text = (
    f"train: {str((OUT_ROOT / 'images' / 'train').as_posix())}\n"
    f"val: {str((OUT_ROOT / 'images' / 'val').as_posix())}\n"
    f"test: {str((OUT_ROOT / 'images' / 'test').as_posix())}\n"
    f"nc: {len(names)}\n"
    "names:\n" + "\n".join([f"  {i}: {n}" for i, n in enumerate(names)])
)
with open(OUT_ROOT / "data.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_text)

print(f"Done. Images with labels: {n}. data.yaml saved to {OUT_ROOT/'data.yaml'}")


Done. Images with labels: 3008. data.yaml saved to datasets\padchest_yolo\data.yaml


In [2]:
# Check GPU
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [1]:
import torch, torchvision
print("torch:", torch.__version__, "| cuda:", torch.version.cuda)
print("torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


torch: 2.5.1+cu121 | cuda: 12.1
torchvision: 0.20.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [1]:
# Step 1: Set environment variable FIRST (before any imports)
import os
os.environ['WANDB_DISABLED'] = 'true'
os.environ['WANDB_MODE'] = 'disabled'

# Step 2: Now import everything else
import torch
from ultralytics import YOLO

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

def on_fit_epoch_end(trainer):
    """Callback to print training metrics at the end of each epoch"""
    try:
        m = trainer.metrics
        
        # Get validation metrics (these are available after validation)
        map50 = m.get('metrics/mAP50(B)', None) or getattr(m, 'map50', None)
        map50_95 = m.get('metrics/mAP50-95(B)', None) or getattr(m, 'map', None)
        
        # Get training loss from the label_loss_items
        # Format: [box_loss, cls_loss, dfl_loss]
        if hasattr(trainer, 'label_loss_items') and trainer.label_loss_items is not None:
            losses = trainer.label_loss_items()
            box_loss = losses[0] if len(losses) > 0 else float('nan')
            cls_loss = losses[1] if len(losses) > 1 else float('nan')
            dfl_loss = losses[2] if len(losses) > 2 else float('nan')
        else:
            box_loss = cls_loss = dfl_loss = float('nan')
        
        print(
            f"Epoch {trainer.epoch + 1}/{trainer.epochs} | "
            f"Train - box:{box_loss:.4f} cls:{cls_loss:.4f} dfl:{dfl_loss:.4f} | "
            f"Val - mAP50:{(map50 if map50 is not None else float('nan')):.4f} "
            f"mAP50-95:{(map50_95 if map50_95 is not None else float('nan')):.4f}"
        )
    except Exception as e:
        print(f"Callback error: {e}")

model = YOLO("yolov8m.pt")
model.add_callback("on_fit_epoch_end", on_fit_epoch_end)

results = model.train(
    data="datasets/padchest_yolo/data.yaml",
    epochs=40,            # fewer epochs
    patience=10,          # early stopping if no improvement for 10 epochs
    imgsz=640,
    batch=8,              # adjust for VRAM
    device=0,             # use first GPU
    workers=8,
    optimizer="AdamW",
    lr0=1e-3,
    cos_lr=True,
    amp=True,
    seed=0,
    deterministic=True,
    compile=False         # skip torch.compile to avoid dynamo issues
)

CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Ultralytics 8.3.204  Python-3.12.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=datasets/padchest_yolo/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train26, nbs=64, nms=False, opset=None, optimize

In [4]:
from ultralytics import YOLO

run_dir = "detect/train26"          # adjust to your folder
model   = YOLO(f"{run_dir}/weights/last.pt")   # use last.pt if best.pt not present

metrics = model.val()
print(metrics)


Ultralytics 8.3.204  Python-3.12.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Model summary (fused): 92 layers, 25,928,926 parameters, 0 gradients, 79.2 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1531.4223.0 MB/s, size: 6502.0 KB)
val: Scanning D:\Projects\projet_tuteuré\datasets\padchest_yolo\labels\val.cache... 300 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300  0.0sScanning D:\Projects\projet_tuteuré\datasets\padchest_yolo\labels\val.cache... 300 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300 126.5Kit/s 0.0s
val: D:\Projects\projet_tuteur\datasets\padchest_yolo\images\val\290116899728494592837689617318363239261_uhft0t.png: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 0.5it/s 37.1s2.5ss
                   all        300        787      0.496      0.164      0.166     0.0709
              NSG tube          8          8    